# 🚀 FoadsIA – Backend Production (GPU T4)

Ejecuta el **backend completo de FoadsIA** en Google Colab usando GPU **NVIDIA T4**.

## 📋 Instrucciones
1. Runtime → Change runtime type → GPU (T4)
2. Ejecuta las celdas **en orden**
3. Copia la URL pública de **ngrok** para tu frontend

---

In [ ]:
# 🛠️ PASO 1 – Configuración del Entorno
import os, subprocess
from pathlib import Path

REPO_URL = "https://github.com/Juanpalojime/FoadsIA.git"
BASE_DIR = Path("/content")
REPO_DIR = BASE_DIR / "FoadsIA"
BACKEND_DIR = REPO_DIR / "backend"

def run(cmd): subprocess.run(cmd, shell=True, check=True)

os.chdir(BASE_DIR)

if not REPO_DIR.exists():
    run(f"git clone {REPO_URL}")
else:
    os.chdir(REPO_DIR)
    run("git pull")

os.chdir(BACKEND_DIR)

run("apt-get update -qq && apt-get install -y -qq ffmpeg libgl1")
run("pip install -q -r requirements.txt")

run("pip install -q flask flask-socketio flask-cors eventlet pyngrok torch torchvision diffusers safetensors openai-whisper insightface onnxruntime-gpu accelerate xformers")

import torch
print('GPU:', torch.cuda.get_device_name(0) if torch.cuda.is_available() else 'NO GPU')


In [ ]:
# ⬇️ PASO 2 – Descarga de Modelos (Master Loader)
import os
from pathlib import Path

os.chdir('/content/FoadsIA/backend')

if Path('master_loader.py').exists():
    !python master_loader.py
else:
    print('❌ No se encontró master_loader.py')


In [ ]:
# 🚀 PASO 3 – Ejecutar Backend + Ngrok
from pyngrok import ngrok
import os, torch, gc

NGROK_TOKEN = ""  # coloca aquí tu token
PORT = 5000

if NGROK_TOKEN:
    ngrok.set_auth_token(NGROK_TOKEN)

ngrok.kill()
public_url = ngrok.connect(PORT, bind_tls=True).public_url

print('🌐 Backend público:', public_url)

torch.cuda.empty_cache()
gc.collect()

os.chdir('/content/FoadsIA/backend')
!python app.py


---
## ✅ Backend FoadsIA listo

- Copia la URL de ngrok
- Conéctala a tu frontend
- No cierres esta pestaña

**FoadsIA Team 🚀**